### Skims & Person-trip Joint
**Author:** Carlos Guirado
**Date:** Feb 2023
**Objective:** Joining stacked daataset and skim wait times

In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil
import io

In [2]:
import pyreadr
import openmatrix as omx

Step 1: path to OMX

In [3]:
#s3_file_key = 'pilates-outputs/sfbay-rh-price-0.000-20221224/activitysim/data/skims.omx'
#bucket = 'beam-outputs'

#s3 = boto3.client('s3')
#obj = s3.get_object(Bucket=bucket, Key=s3_file_key)

#myfile = omx.open_file(io.BytesIO(obj['Body'].read()))

In [4]:
#https://s3.us-east-2.amazonaws.com/beam-outputs/index.html#pilates-outputs/sfbay-baseline-2022124/activitysim/data/
#key = "pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/activitysim/data/skims.omx"


In [5]:
#s3 = boto3.client("s3")
#obj = s3.get_object(Bucket="beam-outputs", Key=key)

Assumes skims have been downloaded locally

In [6]:
#key = r"C:\Users\Administrator\Downloads\base-skims.omx"

In [7]:
row=1 # convention in spreadsheet

In [8]:
print('Reading myfile.omx')
myfile = omx.open_file(f'C:/Users/Administrator/Downloads/skims-2023-{row}.omx')

print ('Shape:', myfile.shape())                 # (100,100)
print ('Number of tables:', len(myfile))         # 3
print ('Table names:', myfile.list_matrices())   # ['m1','m2',',m3']

Reading myfile.omx
Shape: (1454, 1454)
Number of tables: 1263
Table names: ['DIST', 'DISTBIKE', 'DISTWALK', 'DRV_COM_WLK_BOARDS__AM', 'DRV_COM_WLK_BOARDS__EA', 'DRV_COM_WLK_BOARDS__EV', 'DRV_COM_WLK_BOARDS__MD', 'DRV_COM_WLK_BOARDS__PM', 'DRV_COM_WLK_DDIST__AM', 'DRV_COM_WLK_DDIST__EA', 'DRV_COM_WLK_DDIST__EV', 'DRV_COM_WLK_DDIST__MD', 'DRV_COM_WLK_DDIST__PM', 'DRV_COM_WLK_DTIM__AM', 'DRV_COM_WLK_DTIM__EA', 'DRV_COM_WLK_DTIM__EV', 'DRV_COM_WLK_DTIM__MD', 'DRV_COM_WLK_DTIM__PM', 'DRV_COM_WLK_FAR__AM', 'DRV_COM_WLK_FAR__EA', 'DRV_COM_WLK_FAR__EV', 'DRV_COM_WLK_FAR__MD', 'DRV_COM_WLK_FAR__PM', 'DRV_COM_WLK_FERRYIVT__AM', 'DRV_COM_WLK_FERRYIVT__EA', 'DRV_COM_WLK_FERRYIVT__EV', 'DRV_COM_WLK_FERRYIVT__MD', 'DRV_COM_WLK_FERRYIVT__PM', 'DRV_COM_WLK_IVT__AM', 'DRV_COM_WLK_IVT__EA', 'DRV_COM_WLK_IVT__EV', 'DRV_COM_WLK_IVT__MD', 'DRV_COM_WLK_IVT__PM', 'DRV_COM_WLK_IWAIT__AM', 'DRV_COM_WLK_IWAIT__EA', 'DRV_COM_WLK_IWAIT__EV', 'DRV_COM_WLK_IWAIT__MD', 'DRV_COM_WLK_IWAIT__PM', 'DRV_COM_WLK_KEYIVT__A

In [9]:
#m1 = myfile['DIST']
#df1 = pd.DataFrame(m1)
#df1.head()

In [10]:
df = pd.read_csv(r"C:\Users\Administrator\Documents\Carlos\skim_vars.txt", sep=',')

In [11]:
#df.head()

In [12]:
list_vars = list(df.columns.values.tolist())

In [13]:
#list_vars

In [14]:
def grep(l,s):
    return [i for i in l if s in i]

In [15]:
grep(list_vars, 'WAIT')

[" 'DRV_COM_WLK_IWAIT__AM'",
 " 'DRV_COM_WLK_IWAIT__EA'",
 " 'DRV_COM_WLK_IWAIT__EV'",
 " 'DRV_COM_WLK_IWAIT__MD'",
 " 'DRV_COM_WLK_IWAIT__PM'",
 " 'DRV_COM_WLK_WAIT__AM'",
 " 'DRV_COM_WLK_WAIT__EA'",
 " 'DRV_COM_WLK_WAIT__EV'",
 " 'DRV_COM_WLK_WAIT__MD'",
 " 'DRV_COM_WLK_WAIT__PM'",
 " 'DRV_COM_WLK_XWAIT__AM'",
 " 'DRV_COM_WLK_XWAIT__EA'",
 " 'DRV_COM_WLK_XWAIT__EV'",
 " 'DRV_COM_WLK_XWAIT__MD'",
 " 'DRV_COM_WLK_XWAIT__PM'",
 " 'DRV_EXP_WLK_IWAIT__AM'",
 " 'DRV_EXP_WLK_IWAIT__EA'",
 " 'DRV_EXP_WLK_IWAIT__EV'",
 " 'DRV_EXP_WLK_IWAIT__MD'",
 " 'DRV_EXP_WLK_IWAIT__PM'",
 " 'DRV_EXP_WLK_WAIT__AM'",
 " 'DRV_EXP_WLK_WAIT__EA'",
 " 'DRV_EXP_WLK_WAIT__EV'",
 " 'DRV_EXP_WLK_WAIT__MD'",
 " 'DRV_EXP_WLK_WAIT__PM'",
 " 'DRV_EXP_WLK_XWAIT__AM'",
 " 'DRV_EXP_WLK_XWAIT__EA'",
 " 'DRV_EXP_WLK_XWAIT__EV'",
 " 'DRV_EXP_WLK_XWAIT__MD'",
 " 'DRV_EXP_WLK_XWAIT__PM'",
 " 'DRV_HVY_WLK_IWAIT__AM'",
 " 'DRV_HVY_WLK_IWAIT__EA'",
 " 'DRV_HVY_WLK_IWAIT__EV'",
 " 'DRV_HVY_WLK_IWAIT__MD'",
 " 'DRV_HVY_WLK_IWAIT__P

In [16]:
# filter down by each specific time of day (AM, EV, MD, PM)
# narrow down ridehail for a specific ridehail

First set: ridehail pooled in all time periods

In [17]:
time_periods = ['AM', 'PM', 'MD', 'EV', 'EA']
time_periods_solo=[]
time_periods_pool=[]
for element in time_periods:
    i = element + "_WAIT_RH_SOLO"
    j = element + "_WAIT_RH_POOL"
    time_periods_solo.append(i)
    time_periods_pool.append(j)

In [18]:
myfile[f'RH_POOLED_WAIT__AM']

/data/RH_POOLED_WAIT__AM (CArray(1454, 1454)shuffle, zlib(1)) ''
  atom := Float32Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (11, 1454)

In [19]:
print(time_periods_pool)

['AM_WAIT_RH_POOL', 'PM_WAIT_RH_POOL', 'MD_WAIT_RH_POOL', 'EV_WAIT_RH_POOL', 'EA_WAIT_RH_POOL']


In [20]:
df_rh_pool=[]
for elem in time_periods:
    temp = myfile[f'RH_POOLED_WAIT__{elem}'][:,0]
    #print(temp)
    df_rh_pool.append(temp)

df_rh_pool = pd.DataFrame(df_rh_pool)
df_rh_pool = df_rh_pool.T
df_rh_pool.columns = time_periods_pool

In [21]:
df_rh_pool

,AM_WAIT_RH_POOL,PM_WAIT_RH_POOL,MD_WAIT_RH_POOL,EV_WAIT_RH_POOL,EA_WAIT_RH_POOL
0,0.390832,1.652756,0.430667,3.565358,0.333333
1,6.000000,6.000000,6.000000,6.000000,6.000000
2,0.518967,1.156139,0.640219,3.555990,0.591667
3,0.426288,1.682100,0.565671,2.994929,0.400000
4,0.442790,1.380151,0.676597,3.220467,0.850000
...,...,...,...,...,...
1449,3.020624,3.137578,4.122224,3.530862,2.166667
1450,5.241177,2.637774,4.030596,4.548333,4.235717
1451,3.918672,3.729123,5.387678,5.179379,4.316667
1452,6.000000,6.000000,0.450000,6.000000,6.000000


In [22]:
df_rh_pool['TAZ'] = df_rh_pool.index + 1

In [23]:
#df_rh_pool['Max'] = df_rh_pool.idxmax(axis=1)
#df_rh_pool['Max'].value_counts()

In [24]:
df_rh_solo=[]
for elem in time_periods:
    temp = myfile[f'RH_SOLO_WAIT__{elem}'][:,0]
    #print(temp)
    df_rh_solo.append(temp)

df_rh_solo = pd.DataFrame(df_rh_solo)
df_rh_solo = df_rh_solo.T
df_rh_solo.columns = time_periods_solo

In [25]:
df_rh_solo['TAZ'] = df_rh_solo.index + 1

In [26]:
#df_rh_solo['Max'] = df_rh_solo.idxmax(axis=1)
#df_rh_solo['Max'].value_counts()

In [27]:
# read in person trip level data
key ="pilates-outputs/sfbay_baseline_30pct_20230218/inexus/sfbay_baseline_default-1.0_2019__20230217.csv.gz"

s3 = boto3.client("s3")
obj = s3.get_object(Bucket="beam-outputs", Key=key)
df = pd.read_csv(obj['Body'], compression ='gzip')

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_3996\1559954917.py:5: DtypeWarning: Columns (27,77,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,153,162,167,169,170,171,172,174,175,232,233,234,235,244) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(obj['Body'], compression ='gzip')


In [28]:
df.head()

,Unnamed: 0,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,...,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,income_quartiles,income_deciles
0,0,4,1513,57428.0,57078.0,350.0,0.017259,othdiscr,Home,138.0,...,-999.931369,-1998.428483,-1998.428483,-1998.428483,-1998.428483,-5.829695,-7.210364,-5.210327,2ndQ,4thD
1,1,4,1517,58012.0,57733.0,279.0,0.035066,social,othdiscr,48.0,...,-1000.367078,-1998.428483,-1998.428483,-1998.428483,-1998.428483,-8.806431,-10.417154,-6.008901,2ndQ,4thD
2,2,4,1518,60005.0,59475.0,530.0,0.043094,Home,social,158.0,...,-1998.402928,-1998.402928,-1998.402928,-1998.402928,-1998.402928,-9.446440,-8.569069,-6.293542,2ndQ,4thD
3,3,8,2889,61064.0,60199.0,865.0,0.061119,shopping,Home,219.0,...,-999.542207,-1998.428595,-1998.428595,-1998.428595,-1998.428595,-13.654755,-12.696459,-8.158225,2ndQ,4thD
4,4,8,2893,62144.0,61214.0,930.0,0.045421,Home,shopping,296.0,...,-1998.428595,-1998.428595,-1998.428595,-1998.428595,-1998.428595,-12.458058,-12.696883,-7.163804,2ndQ,4thD


In [29]:
list_cols=list(df.columns)

In [30]:
grep(list_cols, 'origin')

['origin_x', 'origin_y']

In [31]:
print(list_cols)

['Unnamed: 0', 'IDMerged', 'tripIndex', 'actStartTime', 'actEndTime', 'duration_travelling', 'cost_BEAM', 'actStartType', 'actEndType', 'duration_walking', 'duration_in_privateCar', 'duration_on_bike', 'duration_in_ridehail', 'distance_travelling', 'duration_in_transit', 'distance_walking', 'distance_bike', 'distance_ridehail', 'distance_privateCar', 'distance_transit', 'vehicleIds_estimate', 'mode_choice_planned_BEAM', 'mode_choice_actual_BEAM', 'vehicleIds', 'numPassengers', 'distance_mode_choice', 'replanning_status', 'reason', 'parkingType', 'transit_bus', 'transit_subway', 'transit_tram', 'transit_cable_car', 'ride_hail_pooled', 'transit_rail', 'fuelFood', 'fuelElectricity', 'fuelBiodiesel', 'fuelDiesel', 'fuel_not_Food', 'fuelGasoline', 'fuel_marginal', 'BlockGroupStart', 'startX', 'startY', 'bgid_start', 'tractid_start', 'juris_name_start', 'county_name_start', 'mpo_start', 'BlockGroupEnd', 'endX', 'endY', 'bgid_end', 'tractid_end', 'juris_name_end', 'county_name_end', 'mpo_end'

In [32]:
df['origin_x'].max()

1454.0

In [33]:
# ASSUME TAZ_X AND TAZ_Y ARE THE VARS TO USE FOR THE JOIN
df=df.merge(df_rh_pool, left_on='origin_x', right_on='TAZ', how='left')

In [34]:
df=df.merge(df_rh_solo, left_on='origin_x', right_on='TAZ', how='left')

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_3996\1378122774.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TAZ_x'} in the result is deprecated and will raise a MergeError in a future version.
  df=df.merge(df_rh_solo, left_on='origin_x', right_on='TAZ', how='left')


In [35]:
df.to_csv('person-trip-with-waits-v2.csv.gz',compression='gzip')